In [ ]:
out_dir = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ad_yeo/results/notebook_07/trained_on_adni'

# 01 - Generate the Correlation Maps for Training

Enter the path to the dictionary generated by Notebook 01

In [ ]:
dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted-avg_vs_explains-most-variance-in-training-data/ad.json'

What correlation method would you like to use?
- Pearson is very fast, but depends on linearity and is sensitive to outliers
- Spearman is slower, but much more robust

In [ ]:
correlation = 'spearman'

Option A - Generate R-maps to optimize 

In [ ]:
# from calvin_utils.ccm_utils.npy_utils import DataLoader
# from calvin_utils.ccm_utils.stat_utils import CorrelationCalculator
# data_loader = DataLoader(dict_path)
# correlation_calculator = CorrelationCalculator(method=correlation, verbose=False)
# corr_map_dict = correlation_calculator.generate_correlation_maps(data_loader)

Option B - Import a-priori maps for optimization

In [ ]:
nifti_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/published_networks/Canonical Networks (Yeo 2011)/*'
mask_path = '/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii'

In [ ]:
from calvin_utils.ccm_utils.nifti_utils import DatasetNiftiImporter
corr_map_dict = DatasetNiftiImporter.import_niftis_to_dict(mask_path, nifti_path)
corr_map_dict

# 02 - Get the Data for Testing

In [ ]:
testing_dict_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/ccm_memory/results/notebook_07/best_maps/weighted-avg_vs_explains-most-variance-in-training-data/ad.json'

In [ ]:
from calvin_utils.ccm_utils.npy_utils import DataLoader
test_data_loader = DataLoader(testing_dict_path)

# 03 - Run Optimizer

Generate the Convergent Maps and Save the Individual Correlation Maps

In [ ]:
from calvin_utils.ccm_utils.optimization import NiftiOptimizer
optimizer = NiftiOptimizer(corr_map_dict, test_data_loader, load_in_time=False)
optimal_map = optimizer.optimize(store_iters=False, store_best=True)
print(optimizer.W)

Save Final Map

In [ ]:
from calvin_utils.ccm_utils.ccm_utils import save_individual_r_maps
save_individual_r_maps(data=optimal_map, out_dir=out_dir, filename='optimized_convergent_map_converged.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save Best Map
- Must set store_best=True

In [ ]:
best_map = optimizer._converge_maps(W=optimizer.best_weights)
from calvin_utils.ccm_utils.ccm_utils import save_individual_r_maps
save_individual_r_maps(data=optimal_map, out_dir=out_dir, filename='optimized_convergent_map_best.nii.gz', mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii', verbose=True)

Save All Intermediate Maps
- Must have set store_iters=True

In [ ]:
for i, vol in enumerate(optimizer.iter_maps):
    save_individual_r_maps(data=optimal_map, 
                           out_dir=out_dir, 
                           filename=f"iteration_maps/iter_{i:03d}.nii.gz", 
                           mask_path='/Users/cu135/hires_backdrops/MNI/MNI152_T1_2mm_brain_mask.nii',
                           verbose=False)

Thats all there is to it

-Calvin